# Import Needed Package


In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load Dataset

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
data = pd.read_csv("/content/drive/MyDrive/Lab - AI /ai-course/Pertemuan-10-Sentiment Analysis-LSTM/IMDB Dataset.csv")

In [10]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
data.shape

(50000, 2)

# Data Encode

In [12]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

# Data Preprocessing

In [14]:
!pip install tensorflow

In [15]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [16]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [17]:
train_data.shape

(40000, 2)

In [18]:
test_data.shape

(10000, 2)

In [19]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data['review'])

#tokenizer untuk bikin kamus kata, untuk tentukan jumlahnya berarti akan cari 5000 kata dalam data yang akan dimasukkan kedalam kamus.

In [23]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

In [24]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [25]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [26]:
Y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


# Model Building

In [27]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid')) #untuk keluarin prediksi

In [28]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [29]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
model.fit(X_train, Y_train, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 222s 425ms/step - accuracy: 0.7260 - loss: 0.5262 - val_accuracy: 0.8372 - val_loss: 0.3662
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 208s 416ms/step - accuracy: 0.8422 - loss: 0.3803 - val_accuracy: 0.8586 - val_loss: 0.3489
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 206s 413ms/step - accuracy: 0.8735 - loss: 0.3084 - val_accuracy: 0.8706 - val_loss: 0.3300
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 206s 412ms/step - accuracy: 0.8910 - loss: 0.2742 - val_accuracy: 0.8727 - val_loss: 0.3104
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 209s 418ms/step - accuracy: 0.9108 - loss: 0.2294 - val_accuracy: 0.8758 - val_loss: 0.3202


In [31]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 38s 118ms/step - accuracy: 0.8798 - loss: 0.3043


In [32]:
print(loss)

0.30606409907341003


In [33]:
print(accuracy)

0.878600001335144


# Building Predictive System

In [39]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequences = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequences)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [40]:
predictive_system("This movie was fantastic and amazing")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step


'positive'

In [41]:
predictive_system("A trilling adventure with stunning visual")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step


'positive'

In [42]:
predictive_system("A visual masterpiece")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step


'positive'

In [43]:
predictive_system ("Overall long and slow")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step


'positive'

In [44]:
model.save("model.h5")

In [45]:
import joblib
joblib.dump(tokenizer, 'tokenizer.pkl')

['tokenizer.pkl']